# Data Preparation

<img src="https://octodex.github.com/images/privateinvestocat.jpg" alt="Kit" title="Cat" width="350" height="200" />
*(image from octodex github)*


In [283]:
#Dataframe packages
import zipfile
import json
import objectpath
import pandas as pd
import numpy as np
import os
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np
from collections import Counter

# Text Mining
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, NMF, PCA

#Widget package
import ipywidgets as wg

# Plot packages
import seaborn as sns
import matplotlib.pyplot as plt

#Oversampling
from imblearn.over_sampling import RandomOverSampler

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexandra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [284]:
name= wg.Text(value='train')
display(name)

Text(value='train')

## Datasets Presentation

The data is provided by [Petfinder.my](https://www.kaggle.com/c/petfinder-adoption-prediction) , a platform dedicated for pets adoption. 
The objective is to predict at which speed a pet is adopted. 

There are 6 different sources of data + Images + Metadata Images and Sentiment Data 
-   Train.csv
-   Test.csv
-   breed_labels.csv
-   color_labels.csv
-   state_labels.csv
-   Images (zip file) from cats and dogs that are adopted
-   Metadata Images (zip file) information about the Image using Google Vision API
-   Sentiment Data is based on the Descriptions using Google's Natural Language API. 


<img src="https://i.imgur.com/i10SHmQ.jpg" alt="K" title="Datasets" width="700" height="400" />

## Data Extraction

### Sentiment Data Magnitude & Score

In [285]:
# Define Empty lists
 
score=[]
magnitude=[]
petid=[]

string = name.value + "_sentiment.zip"
# Read Zip File and Export a Dataset with the Score and the ID
filepath= os.path.join(r"C:\Users\alexandra\Documents\Kaggle", string)

with zipfile.ZipFile(filepath, "r") as z: #Read Zip File
   for filename in z.namelist():  #Filename
      with z.open(filename) as f:  #Open Filename
         data = f.read()  
         d = json.loads(data.decode("utf-8"))
         json_tree = objectpath.Tree(d['documentSentiment'])
         result_tuple = tuple(json_tree.execute('$..score')) #Object Value
         result_tuple2=tuple(json_tree.execute('$..magnitude')) #Object Value
         score.append(result_tuple)
         magnitude.append(result_tuple2)
         
      petid.append(filename.replace('.json',''))

# Output with sentiment data for each pet
sentimental_analysis = pd.concat([ pd.DataFrame(petid, columns =['PetID']) ,pd.DataFrame(score, columns =['sentiment_document_score']),
                                                pd.DataFrame(magnitude, columns =['sentiment_document_magnitude'])],axis =1)

### Sentiment Data Entities Info

In [286]:
 # Define Empty lists
entities=[]
petid=[]

with zipfile.ZipFile(filepath, "r") as z:
   for filename in z.namelist():  
      with z.open(filename) as f:  
         data = f.read()  
         d = json.loads(data.decode("utf-8"))
         json_tree = objectpath.Tree(d['entities'])
         result_tuple = tuple(json_tree.execute('$..name'))
 
         entities.append(result_tuple)
         
      petid.append(filename.replace('.json',''))


entities = pd.DataFrame(entities) #Transform to DataFrame
entities['sentiment_entities_name'] = pd.Series(entities.fillna('').values.tolist()).str.join(' ') # Concatenate Columns

# Output with sentiment data for each pet
sentimental_entities = pd.concat([ pd.DataFrame(petid, columns =['PetID']) ,entities.loc[:, ['sentiment_entities_name']]],axis =1)


### Metadata

In this step we will export the description and the topicality of each image. For more information on [Google API Vision](https://cloud.google.com/vision/)

<img src="https://i.imgur.com/7Hc9gXQ.png" alt="Dogplant" title="Dogplant" width="500" height="600" />
As example in this image the dog is not on the foreground. The Google API doesn't detect the dog.


In [287]:
# Define Empty lists
d = None  
data = None  
description=[]
topicality=[]
imageid=[]


string = name.value + "_metadata.zip"

filepath= os.path.join(r"C:\Users\alexandra\Documents\Kaggle", string)
# Read Zip File and Export a Dataset with the Topicality and Description for each Image
with zipfile.ZipFile(filepath, "r") as z:
   for filename in z.namelist():  
       #Open the Zip File
      with z.open(filename) as f:  
          #Read the json File
          data = f.read()
          d = json.loads(data.decode("utf-8"))
          #Check if the file contains the parent label Annotations and export description and topicality
          if 'labelAnnotations' in d:
             json_tree = objectpath.Tree(d['labelAnnotations'])
             image_metadata1 = list(tuple(json_tree.execute('$..description')))
             image_metadata2 =  list(tuple(json_tree.execute('$..topicality')))

             #Create a list of all descriptions and topicality
             description.append(image_metadata1)
             topicality.append(image_metadata2)
         
             #Create a list with all image id name
             imageid.append(filename.replace('.json',''))


# Prepare the output by renaming all variables
description=pd.DataFrame(description)
topicality=pd.DataFrame(topicality)

#Metadata Description of each image (10 descriptions maximum)
new_names = [(i,'metadata_description_'+str(i)) for i in description.iloc[:, 0:].columns.values]
description.rename(columns = dict(new_names), inplace=True)

#Topicality of the image
new_names = [(i,'metadata_topicality_'+str(i)) for i in topicality.iloc[:, 0:].columns.values]
topicality.rename(columns = dict(new_names), inplace=True)

# Output with sentiment data for each pet
image_metadata = pd.concat([ pd.DataFrame(imageid, columns =['ImageId']) ,topicality,description],axis =1)

# create the PetId variable
image_metadata['PetID'] = image_metadata['ImageId'].str.split('-').str[0]


##############
# TOPICALITY #
##############

#Average
image_metadata['metadata_topicality_mean'] = image_metadata.iloc[:,1:10].mean(axis=1)
image_metadata['metadata_topicality_mean']  = image_metadata.groupby(['PetID'])['metadata_topicality_mean'].transform('mean') 

#Max
image_metadata['metadata_topicality_max'] = image_metadata.iloc[:,1:10].max(axis=1)
image_metadata['metadata_topicality_max'] = image_metadata.groupby(['PetID'])['metadata_topicality_max'].transform(max)

#Minimum
image_metadata['metadata_topicality_min'] = image_metadata.iloc[:,1:10].min(axis=1)
image_metadata['metadata_topicality_min'] = image_metadata.groupby(['PetID'])['metadata_topicality_min'].transform(min)

#Group by Image
image_metadata['metadata_topicality_0_mean']  = image_metadata.groupby(['PetID'])['metadata_topicality_0'].transform('mean')
image_metadata['metadata_topicality_0_max'] = image_metadata.groupby(['PetID'])['metadata_topicality_0'].transform(max)
image_metadata['metadata_topicality_0_min'] = image_metadata.groupby(['PetID'])['metadata_topicality_0'].transform(min)


###############
# DESCRIPTION #
###############
#print(image_metadata)

# Create Features from the Images
image_metadata['L_metadata_0_cat']=image_metadata['metadata_description_0'].str.contains("cat").astype(int)
image_metadata['L_metadata_0_dog'] =image_metadata['metadata_description_0'].str.contains("dog").astype(int)

image_metadata['L_metadata_any_cat']=image_metadata.apply(lambda row: row.astype(str).str.contains('cat').any(), axis=1)
image_metadata['L_metadata_any_dog']=image_metadata.apply(lambda row: row.astype(str).str.contains('dog').any(), axis=1)

image_metadata['L_metadata_0_cat_sum'] = image_metadata.groupby(image_metadata['PetID'])['L_metadata_0_cat'].transform('sum')
image_metadata['L_metadata_0_dog_sum'] = image_metadata.groupby(image_metadata['PetID'])['L_metadata_0_dog'].transform('sum')

image_metadata['L_metadata_any_cat_sum'] = image_metadata.groupby(image_metadata['PetID'])['L_metadata_any_cat'].transform('sum')
image_metadata['L_metadata_any_dog_sum'] = image_metadata.groupby(image_metadata['PetID'])['L_metadata_any_dog'].transform('sum')

image_metadata = image_metadata.fillna("")
image_metadata['metadata_description']= image_metadata[['metadata_description_0', 'metadata_description_1',
                                                        'metadata_description_2', 'metadata_description_3',
                                                        'metadata_description_4', 'metadata_description_5',
                                                        'metadata_description_6', 'metadata_description_7',
                                                        'metadata_description_8', 'metadata_description_9']].apply(lambda x: ' '.join(x), axis=1)
image_metadata = image_metadata[~image_metadata['ImageId'].str.endswith('-1')]
image_metadata = image_metadata[['PetID','metadata_topicality_max','metadata_topicality_mean','metadata_topicality_min',
'metadata_topicality_0_mean','metadata_topicality_0_max','metadata_topicality_0_min','L_metadata_0_cat_sum','L_metadata_0_dog_sum','L_metadata_any_cat_sum','L_metadata_any_dog_sum',
'metadata_description']]
image_metadata=image_metadata.drop_duplicates('PetID')



In [ ]:
image_metadata.columns

Index(['PetID', 'metadata_topicality_max', 'metadata_topicality_mean',
       'metadata_topicality_min', 'metadata_topicality_0_mean',
       'metadata_topicality_0_max', 'metadata_topicality_0_min',
       'L_metadata_0_cat_sum', 'L_metadata_0_dog_sum',
       'L_metadata_any_cat_sum', 'L_metadata_any_dog_sum',
       'metadata_description'],
      dtype='object')

### Metadata Colors

In [ ]:
d = None  
data = None  

color_score_mean=[]
color_score_min=[]
color_score_max=[]

color_pixelfrac_mean=[]
color_pixelfrac_min=[]
color_pixelfrac_max=[]

crops=[]

imageid=[]

# Read Zip File and Export a Dataset with the Score and the ID
with zipfile.ZipFile(filepath, "r") as z:
   for filename in z.namelist():  
      with z.open(filename) as f:  
          data = f.read()
          d = json.loads(data.decode("utf-8"))
          file_keys = list(d.keys())
          if  'imagePropertiesAnnotation' in file_keys:
              file_colors = d['imagePropertiesAnnotation']['dominantColors']['colors']
                        
              file_color_score_mean = np.asarray([x['score'] for x in file_colors]).mean()
              file_color_pixelfrac_mean = np.asarray([x['pixelFraction'] for x in file_colors]).mean()
              
              file_color_score_min = np.asarray([x['score'] for x in file_colors]).min()
              file_color_pixelfrac_min = np.asarray([x['pixelFraction'] for x in file_colors]).min()

              
              file_color_score_max = np.asarray([x['score'] for x in file_colors]).max()
              file_color_pixelfrac_max = np.asarray([x['pixelFraction'] for x in file_colors]).max()
              
              
          #Create a list with all image id name
          imageid.append(filename.replace('.json',''))
          
          color_score_mean.append(file_color_score_mean)
          color_score_min.append(file_color_score_min)
          color_score_max.append(file_color_score_max)
          
          
          color_pixelfrac_mean.append(file_color_pixelfrac_mean)
          color_pixelfrac_min.append(file_color_pixelfrac_min)
          color_pixelfrac_max.append(file_color_pixelfrac_max)

        
image_properties = pd.concat([pd.DataFrame({'ImageId':imageid}),pd.DataFrame({'metadata_color_pixelfrac_mean':color_pixelfrac_mean}), pd.DataFrame({'metadata_color_pixelfrac_min':color_pixelfrac_min}),pd.DataFrame({'metadata_color_pixelfrac_max':color_pixelfrac_max}),pd.DataFrame({'metadata_color_score_mean':color_score_mean}),pd.DataFrame({'metadata_color_score_min':color_score_min}),pd.DataFrame({'metadata_color_score_max':color_score_max})],axis=1)

# create the PetId variable
image_properties['PetID'] = image_properties['ImageId'].str.split('-').str[0]


##############
# COLOR INFO #
##############
image_properties['metadata_color_pixelfrac_mean']  = image_properties.groupby(['PetID'])['metadata_color_pixelfrac_mean'].transform('mean') 
image_properties['metadata_color_pixelfrac_min']  = image_properties.groupby(['PetID'])['metadata_color_pixelfrac_min'].transform(min) 
image_properties['metadata_color_pixelfrac_max']  = image_properties.groupby(['PetID'])['metadata_color_pixelfrac_max'].transform(max) 

image_properties['metadata_color_score_mean']  = image_properties.groupby(['PetID'])['metadata_color_score_mean'].transform('mean') 
image_properties['metadata_color_score_min']  = image_properties.groupby(['PetID'])['metadata_color_score_min'].transform(min) 
image_properties['metadata_color_score_max']  = image_properties.groupby(['PetID'])['metadata_color_score_max'].transform(max) 

image_properties=image_properties.drop_duplicates('PetID')
image_properties = image_properties.drop(['ImageId'], 1)

### Image Quality

Image quality assessment aims to quantitatively represent the human perception of quality. To assign quality images we will add : pixels and  blur score using the variance of Laplacian.  
The following variables are created:
-  Pixel of all images for a pet
-  Pixel average for all pictures for a Pet
-  Blur of all images for a pet
-  Blur average for all pictures for a Pet

<img src="https://i.imgur.com/KjM8TdF.jpg" alt="CatBlur" title="CatBlur" width="200" height="300" />
Example of a blur image.

In [ ]:
def variance_of_laplacian(data):
    image = cv2.imdecode(np.frombuffer(data, np.uint8), 1)      
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)   
    # compute the Laplacian of the image and then return the focus
    # measure, which is simply the variance of the Laplacian
    return cv2.Laplacian(gray, cv2.CV_64F).var() 

blur=[]
image_pixel=[]
imageid =[]

string=name.value + "_images.zip"
filepath= os.path.join(r"C:\Users\alexandra\Documents\Kaggle", string)

#Read the Zip File    
with zipfile.ZipFile(filepath,"r") as zfile:
      for filename in zfile.namelist():
              
              #Blur 
              data = zfile.read(filename)
              # Pixels
              with Image.open( BytesIO(data)) as pixel:
                  width, height = pixel.size
              
              pixel = width*height
              
              #image pixel size for each image
              image_pixel.append(pixel)
              #blur for each image
              blur.append(variance_of_laplacian(data))
              #image id
              imageid.append(filename.replace('.jpg',''))
          
          
# Join Pixel, Blur and Image ID
image_quality = pd.concat([ pd.DataFrame(imageid, columns =['ImageId']) ,pd.DataFrame(blur, columns =['blur']),
                                        pd.DataFrame(image_pixel,columns=['pixel'])],axis =1)
# create the PetId variable
image_quality['PetID'] = image_quality['ImageId'].str.split('-').str[0]

#Mean of the Mean
image1 = image_quality[image_quality['ImageId'].str.endswith('-1')]
image1.columns=['ImageId_1','blur_1','pixel_1','PetID1']

image_quality = image_quality.merge(image1,left_on='PetID', right_on='PetID1')

image_quality['pixel_mean'] = image_quality.groupby(['PetID'])['pixel'].transform('mean')
image_quality['blur_mean'] = image_quality.groupby(['PetID'])['blur'].transform('mean') 

image_quality['pixel_min'] = image_quality.groupby(['PetID'])['pixel'].transform('min') 
image_quality['blur_min'] = image_quality.groupby(['PetID'])['blur'].transform('min')

image_quality['pixel_max'] = image_quality.groupby(['PetID'])['pixel'].transform('max') 
image_quality['blur_max'] = image_quality.groupby(['PetID'])['blur'].transform('max')

image_quality['pixel_sum'] = image_quality.groupby(['PetID'])['pixel'].transform('sum')
image_quality['blur_sum'] = image_quality.groupby(['PetID'])['blur'].transform('sum')


image_quality = image_quality.drop(['blur','pixel','ImageId','ImageId_1'], 1)
image_quality=image_quality.drop_duplicates('PetID')

In [ ]:
print(image_quality)

           PetID       blur_1  pixel_1     PetID1     pixel_mean    blur_mean  \
0      de993d8ad   789.829266   120000  de993d8ad  120000.000000   481.739561   
3      b7142b615   341.153974   120000  b7142b615  120000.000000   304.461764   
11     d6c60a52d    61.693209    20584  d6c60a52d   25951.333333   288.914944   
14     0f6c2d090   167.312757   106400  0f6c2d090  115090.909091   240.007081   
25     e6bc32c73  1326.391723   120000  e6bc32c73  120000.000000  1710.787136   
28     4d8bf7feb  1228.191717   120000  4d8bf7feb  120000.000000  1153.183411   
31     4732c7563   317.842668   172800  4732c7563  172800.000000   295.648802   
35     9d466bbca   294.486610   120000  9d466bbca  115408.909091   604.807373   
46     fe3fe601e  1640.025935   120000  fe3fe601e  120100.000000  1469.094943   
50     d51b2e785   213.602437   120000  d51b2e785  120000.000000   236.029526   
55     e7c607873   121.293109   120000  e7c607873  120000.000000   134.579769   
59     c1ca516d7   697.44364

### Hu moments

In [ ]:
# Read image as grayscale image
from math import copysign, log10
import numpy as np

huMoments0=[]
huMoments1=[]
huMoments2=[]
huMoments3=[]
huMoments4=[]
huMoments5=[]
huMoments6=[]
imageid =[]

string=name.value + "_images.zip"
filepath= os.path.join(r"C:\Users\alexandra\Documents\Kaggle", string)

#Read the Zip File    
with zipfile.ZipFile(filepath,"r") as zfile:
      for filename in zfile.namelist():
            data = zfile.read(filename)
            
            image = cv2.imdecode(np.frombuffer(data, np.uint8), 1)      
            im = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)   
            # Calculate Moments
            moments = cv2.moments(im)
 
            # Calculate Hu Moments
            huMoments = cv2.HuMoments(moments)
            # Log scale hu moments
            for i in range(0,7):
                  huMoments[i] = round(-1* copysign(1.0, huMoments[i]) * log10(abs(huMoments[i])),2)
                
            #image id
            imageid.append(filename.replace('.jpg',''))
            huMoments0.append(huMoments[0])
            
            huMoments1.append(huMoments[1])
            huMoments2.append(huMoments[2])
            huMoments3.append(huMoments[3])
            huMoments4.append(huMoments[4])
            huMoments5.append(huMoments[5])
            huMoments6.append(huMoments[6])

image_moments = pd.concat([pd.DataFrame({'ImageId':imageid}),pd.DataFrame({'huMoments0':np.concatenate(huMoments0,axis=0)}), pd.DataFrame({'huMoments1':np.concatenate(huMoments1,axis=0)}),
                           pd.DataFrame({'huMoments2':np.concatenate(huMoments2,axis=0)}),pd.DataFrame({'huMoments3':np.concatenate(huMoments3,axis=0)}),pd.DataFrame({'huMoments4':np.concatenate(huMoments4,axis=0)}),
                           pd.DataFrame({'huMoments5':np.concatenate(huMoments5,axis=0)}),pd.DataFrame({'huMoments6':np.concatenate(huMoments6,axis=0)})],axis=1)

arr = np.array(data)

# create the PetId variable
image_moments['PetID'] = image_moments['ImageId'].str.split('-').str[0]
image_moments = image_moments[image_moments['ImageId'].apply(lambda x:x.endswith(("-1")))]

## Load Data

In [ ]:
#Load Data

if name.value == "train":
    df =pd.read_csv(r'C:\Users\alexandra\Documents\Kaggle\train.csv')
    
    
if name.value == "test":
    df =pd.read_csv(r'C:\Users\alexandra\Documents\Kaggle\test.csv')

#test = pd.read_csv('https://raw.githubusercontent.com/alexlorenzo/Pet.my/master/test.csv')
breed =pd.read_csv(r'C:\Users\alexandra\Documents\Kaggle\breed_labels.csv',usecols=["BreedID", "BreedName"]) #A pet could have multiple breed
color =pd.read_csv(r'C:\Users\alexandra\Documents\Kaggle\color_labels.csv') #A pet could have multiple colors
state =pd.read_csv(r'C:\Users\alexandra\Documents\Kaggle\state_labels.csv')

# Add information about color, breed, state and sentiment data
df = (pd.merge(df, breed.rename(columns={"BreedName": "BreedName1"}),  how='left', left_on=['Breed1'], right_on = ['BreedID']).drop('BreedID', axis=1))
df = (pd.merge(df, breed.rename(columns={"BreedName": "BreedName2"}),  how='left', left_on=['Breed2'], right_on = ['BreedID']).drop('BreedID', axis=1))

df = (pd.merge(df, color.rename(columns={"ColorName": "ColorName1"}),  how='left', left_on=['Color1'], right_on = ['ColorID']).drop('ColorID', axis=1))
df = (pd.merge(df, color.rename(columns={"ColorName": "ColorName2"}),  how='left', left_on=['Color2'], right_on = ['ColorID']).drop('ColorID', axis=1))
df = (pd.merge(df, color.rename(columns={"ColorName": "ColorName3"}),  how='left', left_on=['Color3'], right_on = ['ColorID']).drop('ColorID', axis=1))
df = (pd.merge(df, state,  how='left', left_on=['State'], right_on = ['StateID']).drop('StateID', axis=1))

df = (pd.merge(df, sentimental_analysis,  how='left', left_on=['PetID'], right_on = ['PetID']))
df = (pd.merge(df, sentimental_entities,  how='left', left_on=['PetID'], right_on = ['PetID']))

# Add information about Metadata Images
df = (pd.merge(df, image_metadata,  how='left', left_on=['PetID'], right_on = ['PetID']))
# Add information about Metadata Images
df = (pd.merge(df, image_properties,  how='left', left_on=['PetID'], right_on = ['PetID']))
df = (pd.merge(df, image_moments,  how='left', left_on=['PetID'], right_on = ['PetID']))

# Add information about quality Images
df= (pd.merge(df, image_quality,  how='left', left_on=['PetID'], right_on = ['PetID']))


In [ ]:
pd.set_option('display.max_columns',100)
df.head(3)

## Features 

### Malaysia Information

In [ ]:
## Using the Kernel:https://www.kaggle.com/bibek777/stacking-kernels

# state GDP: https://en.wikipedia.org/wiki/List_of_Malaysian_states_by_GDP
state_gdp = {
    41336: 116.679,
    41325: 40.596,
    41367: 23.02,
    41401: 190.075,
    41415: 5.984,
    41324: 37.274,
    41332: 42.389,
    41335: 52.452,
    41330: 67.629,
    41380: 5.642,
    41327: 81.284,
    41345: 80.167,
    41342: 121.414,
    41326: 280.698,
    41361: 32.270
}

# state population: https://en.wikipedia.org/wiki/Malaysia
state_population = {
    41336: 33.48283,
    41325: 19.47651,
    41367: 15.39601,
    41401: 16.74621,
    41415: 0.86908,
    41324: 8.21110,
    41332: 10.21064,
    41335: 15.00817,
    41330: 23.52743,
    41380: 2.31541,
    41327: 15.61383,
    41345: 32.06742,
    41342: 24.71140,
    41326: 54.62141,
    41361: 10.35977
}

state_area ={
    41336:19102,
41325:9500,
41367:15099,
41401:243,
41415:91,
41324:1664,
41332:6686,
41335:36137,
41330:21035,
41380:821,
41327:1048,
41345:73631,
41342:124450,
41326:8104,
41361:13035}

state_unemployment ={
    41336 : 3.6,
41325 :2.9,
41367: 3.8,
41324: 0.9,
41332 : 2.7,
41335: 2.6,
41330: 3.4,
41380: 2.9,
41327: 2.1,
41345 : 5.4,
41342 : 3.3,
41326: 3.2,
41361: 4.2,
41415: 7.8,
41401: 3.3
}

# per 1000 population
state_birth_rate = {
 41336:16.3,
41325:17.0,
41367:21.4,
41401:14.4,
41415:18.1,
41324:16.0,
41332:16.4,
41335:17.0,
41330:14.4,
41380:17.5,
41327:12.7,
41345:13.7,
41342:13.9,
41326:16.6,
41361:23.3,     
}

df["state_gdp"] = df.State.map(state_gdp)
df["state_population"] = df.State.map(state_population)
df["state_area"] = df.State.map(state_area)
df['state_unemployment']=df.State.map(state_unemployment)
df['state_birth_rate']=df.State.map(state_birth_rate)

### Features Engineering

In [ ]:
# Color (Create a Flag pet has 1 color, 2 colors, 3 colors)
df['L_Color1'] = (pd.isnull(df['ColorName3']) & pd.isnull(df['ColorName2']) & pd.notnull(df['ColorName1'])).astype(int)
df['L_Color2'] = (pd.isnull(df['ColorName3']) & pd.notnull(df['ColorName2']) & pd.notnull(df['ColorName1'])).astype(int)
df['L_Color3'] = (pd.notnull(df['ColorName3']) & pd.notnull(df['ColorName2']) & pd.notnull(df['ColorName1'])).astype(int)

# Breed (create a flag if the pet has 1 breed or 2)
df['L_Breed1'] = (pd.isnull(df['BreedName2']) & pd.notnull(df['BreedName1'])).astype(int)
df['L_Breed2'] = (pd.notnull(df['BreedName2']) & pd.notnull(df['BreedName1'])).astype(int)

# Breed create columns
df['L_Breed1_Siamese'] =(df['BreedName1']=='Siamese').astype(int)
df['L_Breed1_Persian']=(df['BreedName1']=='Persian').astype(int)
df['L_Breed1_Labrador_Retriever']=(df['BreedName1']=='Labrador Retriever').astype(int)
df['L_Breed1_Terrier']=(df['BreedName1']=='Terrier').astype(int)
df['L_Breed1_Golden_Retriever ']=(df['BreedName1']=='Golden Retriever').astype(int)

#Name (create a flag if the name is missing, with less than two letters)
df['L_Name_missing'] =  (pd.isnull(df['Name'])).astype(int)
df['Name_Length']=df['Name'].str.len() 

#Description 
df['Description_Length']=df['Description'].str.len() 

# Fee Amount
df['L_Fee_Free'] =  (df['Fee']==0).astype(int)

#Add the Number of Pets per Rescuer 
pets_total = df.groupby(['RescuerID']).size().reset_index(name='N_pets_total')
df= pd.merge(df, pets_total, left_on='RescuerID', right_on='RescuerID', how='inner')
df.count()

# No photo
df['L_NoPhoto'] =  (df['PhotoAmt']==0).astype(int)

#No Video
df['L_NoVideo'] =  (df['VideoAmt']==0).astype(int)

#Log Age 
df['Log_Age']= np.log(df.Age+1) 

#Negative Score 
df['L_scoreneg'] =  (df['sentiment_document_score']<0).astype(int)

#Quantity Amount >5
df.loc[df['Quantity'] > 5, 'Quantity'] = 5

### Text Mining Features

In [ ]:
# Normalize the Variable Description
from collections import OrderedDict

def cleaning_text(df,column):  
    df[column] =df[column].fillna("<MISSING>") #Replace missing values
    df[column] = df[column].str.replace('\d+', '') #Remove numbers
    df[column] = df[column].str.lower()
    df[column] = df[column].str.replace('[^\w\s]','') #Remove punctuactions
    df[column] = df[column].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' ') #Delete duplicated word

    stop = stopwords.words('english') #Do not take into account stop words (ex: The)

    pat = r'\b(?:{})\b'.format('|'.join(stop))
    df[column] = df[column].str.replace(pat, '')
    df[column] = df[column].str.replace(r'\s+', ' ')

    # Stem Words
    df[column] = df[column].astype(str).str.split()

    porter_stemmer = PorterStemmer()
    df[column]=df[column].apply(lambda x : [porter_stemmer.stem(y) for y in x]) #Word of the same family

    df[column]=df[column].apply(lambda x : " ".join(x))
    return df

df = cleaning_text(df,'Description')
df = cleaning_text(df,'sentiment_entities_name')
df = cleaning_text (df,'metadata_description')

In [ ]:
# Matrix Factorization for dimensionality reduction
def matrix_factorization(df,column,prefix_svd, prefix_nmf):
    
    svd_ = TruncatedSVD(
        n_components=5, random_state=1337)
    nmf_ = NMF(
        n_components=5, random_state=1337)

    tfidf_col = TfidfVectorizer().fit_transform(df[column])
    svd_col = svd_.fit_transform(tfidf_col)
    svd_col = pd.DataFrame(svd_col)
    svd_col = svd_col.add_prefix(prefix_svd)

    nmf_col = nmf_.fit_transform(tfidf_col)
    nmf_col = pd.DataFrame(nmf_col)
    nmf_col = nmf_col.add_prefix(prefix_nmf)
    df = pd.concat([df,nmf_col,svd_col],axis=1)
    return df
    
df = matrix_factorization(df,'Description','SVD_Desc_','NMF_Desc_')
df = matrix_factorization(df,'sentiment_entities_name','SVD_entities_','NMF_entities_')
df = matrix_factorization(df,'metadata_description','SVD_metadata_desc_','NMF_metadata_desc_')

In [ ]:
df.head()

### Drop Features

In [ ]:
# Cannot be used for this analysis (IDs, Texts...)
#df = df.drop(["Name","Description","BreedName2","ColorName3",'Name','Breed1','Breed2','RescuerID',#'Description',
#              'BreedName1','Color1', 'Color2', 'Color3','Age','State','metadata_description','sentiment_entities_name'
#              'ColorName1','ColorName2','Health', 'Gender', 'Dewormed','Type','MaturitySize', 'Sterilized','Vaccinated','FurLength','StateName'],axis=1)

### Missing Values

In [ ]:
columns = df.columns
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': columns,
                                 'percent_missing': percent_missing})

missing_value_df =missing_value_df[missing_value_df['percent_missing']>0]
missing_value_df


plt.figure(figsize=(15, 8))
ax = sns.barplot(x="column_name", y="percent_missing", data=missing_value_df, label='Sales')
ax.set_xticklabels(ax.get_xticklabels(),rotation=75)
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2., p.get_height(), '%d' % round(p.get_height(),2) + '%', 
            fontsize=12, color='grey', ha='center', va='bottom') 
    
plt.show()

In [ ]:
for col in ['sentiment_document_score', 'sentiment_document_magnitude','metadata_topicality_max','metadata_topicality_mean','metadata_topicality_min',
           'metadata_topicality_0_mean','metadata_topicality_0_max','metadata_topicality_0_min','L_metadata_0_dog_sum',
           'L_metadata_0_cat_sum','L_metadata_any_dog_sum','L_metadata_any_cat_sum','pixel_mean','pixel_min','pixel_max','pixel_sum',
           'blur_min','blur_max','blur_sum','blur_mean','metadata_color_pixelfrac_mean','metadata_color_pixelfrac_min',
           'metadata_color_pixelfrac_max','metadata_color_score_mean','metadata_color_score_min','metadata_color_score_max',
            'Name_Length','Description_Length','huMoments0','huMoments1','huMoments2','huMoments3','huMoments4','huMoments5','huMoments6']:
    

    df[col].fillna((df[col].median()), inplace=True)
    
# replacing na values with No Color 
df["ColorName2"].fillna("No Color", inplace = True) 

### Feature Categorization

In [ ]:
#One Hot Encoding
#df = pd.concat([df.drop('StateName', axis=1),pd.get_dummies(df['StateName'], prefix='State')], axis=1)
df = pd.concat([df,pd.get_dummies(df['StateName'], prefix='State')], axis=1)

col=['ColorName1','ColorName2','Health', 'Gender', 'Dewormed','Type','MaturitySize', 'Sterilized','Vaccinated','FurLength']
for i in col:
    #df = pd.concat([df.drop(i, axis=1),pd.get_dummies(df[i], prefix=i)], axis=1)
    df = pd.concat([df,pd.get_dummies(df[i], prefix=i)], axis=1)

In [ ]:
output= name.value + "_features.csv"
print(output)
filepath= os.path.join(r"C:\Users\alexandra\Documents\Kaggle", output)
df.to_csv(filepath)